<a href="https://colab.research.google.com/github/F-Bafti/AI-Agents-and-Agentic-AI/blob/master/Decorated_Modular_AI_Agent_Coursera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decorated Modular AI Agent Implementation
In our original README agent, we manually created and registered each action. While this approach works, it requires us to maintain the tool definitions (functions) separately from their metadata (descriptions and parameters). This separation creates opportunities for these two pieces to become out of sync. Let’s improve our agent by using tool decorators to keep everything together and automatically synchronized.

In [36]:
!pip install langchain-community
!pip uninstall cohere langchain-cohere -y

!pip install cohere>=5.0.0
!pip install langchain-cohere

!pip install cohere==5.11.0 langchain-cohere==0.3.0

import json, os
from google.colab import userdata

api_key = userdata.get('COHERE_API_KEY')
os.environ['COHERE_API_KEY'] = api_key

# Decorated Agent Tools and Actions Setting

In [27]:
import inspect
from typing import get_type_hints, List, Callable, Dict, Any, Optional

tools = {}
tools_by_tag = {}


def to_openai_tools(tools_metadata: List[dict]):
    cohere_tools = [
        {
            "type": "function",
            "function": {
                "name": t['tool_name'],
                # Include up to 1024 characters of the description
                "description": t.get('description',"")[:1024],
                "parameters": t.get('parameters',{}),
            },
        } for t in tools_metadata
    ]
    return cohere_tools

def get_tool_metadata(func, tool_name=None, description=None, parameters_override=None, terminal=False, tags=None):
    """
    Extracts metadata for a function to use in tool registration.

    Parameters:
        func (function): The function to extract metadata from.
        tool_name (str, optional): The name of the tool. Defaults to the function name.
        description (str, optional): Description of the tool. Defaults to the function's docstring.
        parameters_override (dict, optional): Override for the argument schema. Defaults to dynamically inferred schema.
        terminal (bool, optional): Whether the tool is terminal. Defaults to False.
        tags (List[str], optional): List of tags to associate with the tool.

    Returns:
        dict: A dictionary containing metadata about the tool, including description, args schema, and the function.
    """
    # Default tool_name to the function name if not provided
    tool_name = tool_name or func.__name__

    # Default description to the function's docstring if not provided
    description = description or (func.__doc__.strip() if func.__doc__ else "No description provided.")

    # Discover the function's signature and type hints if no args_override is provided
    if parameters_override is None:
        signature = inspect.signature(func)
        type_hints = get_type_hints(func)

        # Build the arguments schema dynamically
        args_schema = {
            "type": "object",
            "properties": {},
            "required": []
        }
        for param_name, param in signature.parameters.items():

            if param_name in ["action_context", "action_agent"]:
                continue  # Skip these parameters

            def get_json_type(param_type):
                if param_type == str:
                    return "string"
                elif param_type == int:
                    return "integer"
                elif param_type == float:
                    return "number"
                elif param_type == bool:
                    return "boolean"
                elif param_type == list:
                    return "array"
                elif param_type == dict:
                    return "object"
                else:
                    return "string"

            # Add parameter details
            param_type = type_hints.get(param_name, str)  # Default to string if type is not annotated
            param_schema = {"type": get_json_type(param_type)}  # Convert Python types to JSON schema types

            args_schema["properties"][param_name] = param_schema

            # Add to required if not defaulted
            if param.default == inspect.Parameter.empty:
                args_schema["required"].append(param_name)
    else:
        args_schema = parameters_override

    # Return the metadata as a dictionary
    return {
        "tool_name": tool_name,
        "description": description,
        "parameters": args_schema,
        "function": func,
        "terminal": terminal,
        "tags": tags or []
    }


def register_tool(tool_name=None, description=None, parameters_override=None, terminal=False, tags=None):
    """
    A decorator to dynamically register a function in the tools dictionary with its parameters, schema, and docstring.

    Parameters:
        tool_name (str, optional): The name of the tool to register. Defaults to the function name.
        description (str, optional): Override for the tool's description. Defaults to the function's docstring.
        parameters_override (dict, optional): Override for the argument schema. Defaults to dynamically inferred schema.
        terminal (bool, optional): Whether the tool is terminal. Defaults to False.
        tags (List[str], optional): List of tags to associate with the tool.

    Returns:
        function: The wrapped function.
    """
    def decorator(func):
        # Use the reusable function to extract metadata
        metadata = get_tool_metadata(
            func=func,
            tool_name=tool_name,
            description=description,
            parameters_override=parameters_override,
            terminal=terminal,
            tags=tags
        )

        # Register the tool in the global dictionary
        tools[metadata["tool_name"]] = {
            "description": metadata["description"],
            "parameters": metadata["parameters"],
            "function": metadata["function"],
            "terminal": metadata["terminal"],
            "tags": metadata["tags"] or []
        }

        for tag in metadata["tags"]:
            if tag not in tools_by_tag:
                tools_by_tag[tag] = []
            tools_by_tag[tag].append(metadata["tool_name"])

        return func
    return decorator


# Define Prompt Class and Agent Response

In [30]:
import os
from dataclasses import dataclass, field
import json
import time
import asyncio
import traceback
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FuturesTimeoutError

@dataclass
class Prompt:
    messages: List[Dict] = field(default_factory=list)
    tools: List[Dict] = field(default_factory=list)
    metadata: dict = field(default_factory=dict)  # Fixing mutable default issue



def generate_response(prompt: Prompt) -> str:
    """Call LLM to get response"""
    print("DEBUG: Starting generate_response")
    print(f"DEBUG: Prompt has {len(prompt.messages)} messages and {len(prompt.tools)} tools")

    try:
        # Initialize Cohere model with LangChain
        llm = ChatCohere(
            model="command-r-plus",
            max_tokens=1024,
            temperature=0.3,
            timeout=30  # Add timeout
        )
        print("DEBUG: Cohere model initialized")

        messages = prompt.messages
        tools = prompt.tools
        result = None

        # Convert dict messages to LangChain message objects
        langchain_messages = []
        for msg in messages:
            if msg["role"] == "system":
                langchain_messages.append(SystemMessage(content=msg["content"]))
            elif msg["role"] == "user":
                langchain_messages.append(HumanMessage(content=msg["content"]))
            elif msg["role"] == "assistant":
                langchain_messages.append(AIMessage(content=msg["content"]))

        print(f"DEBUG: Converted {len(langchain_messages)} messages")

        # If there is no tools for LLM to use just get the text response
        if not tools:
            print("DEBUG: No tools, making simple invoke call")
            response = llm.invoke(langchain_messages)
            result = response.content
        else:
            print("DEBUG: Tools present, setting up tool calling")

            # Debug: Print the tools before formatting
            print(f"DEBUG: Original tools: {tools}")

            formatted_tools = []
            for tool in tools:
                if isinstance(tool, dict) and "function" in tool:
                    func_def = tool["function"]
                    cohere_tool = {
                        "title": func_def["name"],
                        "description": func_def["description"],
                        "properties": func_def["parameters"].get("properties", {}),
                        "required": func_def["parameters"].get("required", [])
                    }
                    formatted_tools.append(cohere_tool)

            print(f"DEBUG: Formatted tools: {formatted_tools}")

            llm_with_tools = llm.bind_tools(formatted_tools)
            print("DEBUG: About to invoke LLM with tools (this may take 30-60 seconds)")

            # Add timeout wrapper
            def make_call():
                return llm_with_tools.invoke(langchain_messages)

            with ThreadPoolExecutor() as executor:
                future = executor.submit(make_call)
                try:
                    response = future.result(timeout=60)  # 60 second timeout
                    print("DEBUG: Got response from LLM with tools")
                except FuturesTimeoutError:
                    print("DEBUG: LLM call timed out after 60 seconds")
                    return "ERROR: LLM call timed out"

            if response.tool_calls:
                tool_call = response.tool_calls[0]
                result = {
                    "tool": tool_call["name"],
                    "args": tool_call["args"],
                }
                result = json.dumps(result)
            else:
                result = response.content

        print(f"DEBUG: Returning result: {result}")
        return result

    except Exception as e:
        print(f"DEBUG: Exception occurred: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()
        return f"ERROR: {str(e)}"


# Setting Up the GAME(Goal, Action, Memory, Environment)

In [31]:
# 1- Goal Class
# Creates a simple container to hold information about what the agent should accomplish
# priority: How important this goal is (lower numbers = higher priority)
# name: Short name for the goal (like "Gather Information")
# description: Detailed explanation of what to do
# frozen=True: Makes this immutable - once created, it can't be changed
@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str



# 2- Action Class
# Creates a wrapper around a Python function to make it available to the AI
# name: What the AI will call this action (like "read_file")
# function: The actual Python function to execute
# description: Tells the AI what this function does
# parameters: Describes what arguments the function needs (JSON schema format)
# terminal: Whether calling this action should end the agent's execution
class Action:
    def __init__(self,
                 name: str,
                 function: Callable,
                 description: str,
                 parameters: Dict,
                 terminal: bool = False):
        self.name = name
        self.function = function
        self.description = description
        self.terminal = terminal
        self.parameters = parameters

    def execute(self, **args) -> Any:
        """Execute the action's function"""
        return self.function(**args)

# Creates a container to store all available actions
class ActionRegistry:
    def __init__(self):
        self.actions = {}

    def register(self, action: Action):
        self.actions[action.name] = action

    # Looks up an action by its name
    def get_action(self, name: str) -> Action | None:
        return self.actions.get(name, None)

    # Returns ALL actions as a list
    def get_actions(self) -> List[Action]:
        """Get all registered actions"""
        return list(self.actions.values())



# 3- Memory Class
# Creates a container to store the conversation history
# self.items = []: An empty list to hold memory items
# Each item will be a dictionary representing one piece of the conversation
class Memory:
    def __init__(self):
        self.items = []  # Basic conversation history

    # Adds a new memory item to the end of the list
    def add_memory(self, memory: dict):
        """Add memory to working memory"""
        self.items.append(memory)

    # Returns the stored memories as a list
    def get_memories(self, limit: int = None) -> List[Dict]:
        """Get formatted conversation history for prompt"""
        return self.items[:limit]

    # Creates a new Memory object with system messages filtered out
    def copy_without_system_memories(self):
        """Return a copy of the memory without system memories"""
        filtered_items = [m for m in self.items if m["type"] != "system"]
        memory = Memory()
        memory.items = filtered_items
        return memory



# 4- Environment Class
# This is where actions actually get executed safely
# try: attempts to run the action
# action.execute(**args) calls the action with the provided arguments
# If it works: calls self.format_result() to package the result nicely
# If it fails: catches the error and returns error information instead of crashing
class Environment:
    def execute_action(self, action: Action, args: dict) -> dict:
        """Execute an action and return the result."""
        try:
            result = action.execute(**args)
            return self.format_result(result)
        except Exception as e:
            return {
                "tool_executed": False,
                "error": str(e),
                "traceback": traceback.format_exc()
            }

    def format_result(self, result: Any) -> dict:
        """Format the result with metadata."""
        return {
            "tool_executed": True,
            "result": result,
            "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z")
        }


# Setting up the Agent Language and Function-Calling

In [32]:
# What this class do?
# Example: user_input = "Write a README for this project."
# What construct_prompt Does:
# The agent takes that simple user input and builds a complex prompt that includes:
# System instructions (the goals), Conversation history, Available tools, The user's request
# Prompt(
#     messages=[
#         {"role": "system", "content": "Goal: Read each file..."},
#         {"role": "user", "content": "Write a README for this project."},
#         {"role": "assistant", "content": '{"tool": "list_files", "args": {}}'},
#         {"role": "user", "content": "Tool result: [file1.py, file2.py]"}
#     ],
#     tools=[list_files_tool, read_file_tool, terminate_tool]
# )

class AgentLanguage:
    def __init__(self):
        pass

    def construct_prompt(self,
                         actions: List[Action],
                         environment: Environment,
                         goals: List[Goal],
                         memory: Memory) -> Prompt:
        raise NotImplementedError("Subclasses must implement this method")

    def parse_response(self, response: str) -> dict:
        raise NotImplementedError("Subclasses must implement this method")



# This is a concrete implementation of the abstract AgentLanguage class
class AgentFunctionCallingActionLanguage(AgentLanguage):

    def __init__(self):
        super().__init__()

    # Takes a list of Goal objects and converts them into a system message
    # Creates a formatted string with separators to make it readable
    # Returns a list with one system message containing all goals
    def format_goals(self, goals: List[Goal]) -> List:
        # Map all goals to a single string that concatenates their description
        # and combine into a single message of type system
        sep = "\n-------------------\n"
        goal_instructions = "\n\n".join([f"{goal.name}:{sep}{goal.description}{sep}" for goal in goals])
        return [
            {"role": "system", "content": goal_instructions}
        ]


    def format_memory(self, memory: Memory) -> List:
        """Generate response from language model"""
        # Map all environment results to a role:user messages
        # Map all assistant messages to a role:assistant messages
        # Map all user messages to a role:user messages
        items = memory.get_memories()
        mapped_items = []
        for item in items:

            content = item.get("content", None)
            if not content:
                content = json.dumps(item, indent=4)

            if item["type"] == "assistant":
                mapped_items.append({"role": "assistant", "content": content})
            elif item["type"] == "environment":
                # Map environment results to user messages for Cohere compatibility
                mapped_items.append({"role": "user", "content": f"Tool result: {content}"})
            else:
                mapped_items.append({"role": "user", "content": content})

        return mapped_items

    def format_actions(self, actions: List[Action]) -> List:
        """Convert actions to LangChain-compatible tool format"""

        tools = []
        for action in actions:
            # Convert to OpenAI function format that LangChain can use
            tool_def = {
                "type": "function",
                "function": {
                    "name": action.name,
                    "description": action.description[:1024],
                    "parameters": action.parameters,
                }
            }
            tools.append(tool_def)

        return tools

    def construct_prompt(self,
                         actions: List[Action],
                         environment: Environment,
                         goals: List[Goal],
                         memory: Memory) -> Prompt:

        prompt = []
        prompt += self.format_goals(goals)
        prompt += self.format_memory(memory)

        tools = self.format_actions(actions)

        return Prompt(messages=prompt, tools=tools)

    def adapt_prompt_after_parsing_error(self,
                                         prompt: Prompt,
                                         response: str,
                                         traceback: str,
                                         error: Any,
                                         retries_left: int) -> Prompt:

        return prompt

    def parse_response(self, response: str) -> dict:
        """Parse LLM response into structured format by extracting the ```json block"""

        try:
            return json.loads(response)

        except Exception as e:
            return {
                "tool": "terminate",
                "args": {"message": response}
            }


class PythonActionRegistry(ActionRegistry):
    def __init__(self, tags: List[str] = None, tool_names: List[str] = None):
        super().__init__()

        self.terminate_tool = None

        for tool_name, tool_desc in tools.items():
            if tool_name == "terminate":
                self.terminate_tool = tool_desc

            if tool_names and tool_name not in tool_names:
                continue

            tool_tags = tool_desc.get("tags", [])
            if tags and not any(tag in tool_tags for tag in tags):
                continue

            self.register(Action(
                name=tool_name,
                function=tool_desc["function"],
                description=tool_desc["description"],
                parameters=tool_desc.get("parameters", {}),
                terminal=tool_desc.get("terminal", False)
            ))

    def register_terminate_tool(self):
        if self.terminate_tool:
            self.register(Action(
                name="terminate",
                function=self.terminate_tool["function"],
                description=self.terminate_tool["description"],
                parameters=self.terminate_tool.get("parameters", {}),
                terminal=self.terminate_tool.get("terminal", False)
            ))
        else:
            raise Exception("Terminate tool not found in tool registry")


# The Agent Class

In [33]:
class Agent:
    def __init__(self,
                 goals: List[Goal],
                 agent_language: AgentLanguage,
                 action_registry: ActionRegistry,
                 generate_response: Callable[[Prompt], str],
                 environment: Environment):
        """
        Initialize an agent with its core GAME components
        """
        self.goals = goals
        self.generate_response = generate_response
        self.agent_language = agent_language
        self.actions = action_registry
        self.environment = environment

    def construct_prompt(self, goals: List[Goal], memory: Memory, actions: ActionRegistry) -> Prompt:
        """Build prompt with memory context"""
        return self.agent_language.construct_prompt(
            actions=actions.get_actions(),
            environment=self.environment,
            goals=goals,
            memory=memory
        )

    def get_action(self, response):
        invocation = self.agent_language.parse_response(response)
        action = self.actions.get_action(invocation["tool"])
        return action, invocation

    def should_terminate(self, response: str) -> bool:
        action_def, _ = self.get_action(response)
        return action_def.terminal

    def set_current_task(self, memory: Memory, task: str):
        memory.add_memory({"type": "user", "content": task})

    def update_memory(self, memory: Memory, response: str, result: dict):
        """
        Update memory with the agent's decision and the environment's response.
        """
        new_memories = [
            {"type": "assistant", "content": response},
            {"type": "environment", "content": json.dumps(result)}
        ]
        for m in new_memories:
            memory.add_memory(m)

    def prompt_llm_for_action(self, full_prompt: Prompt) -> str:
        response = self.generate_response(full_prompt)
        return response

    def run(self, user_input: str, memory=None, max_iterations: int = 50) -> Memory:
        """
        Execute the GAME loop for this agent with a maximum iteration limit.
        """
        memory = memory or Memory()
        self.set_current_task(memory, user_input)

        for _ in range(max_iterations):
            # Construct a prompt that includes the Goals, Actions, and the current Memory
            prompt = self.construct_prompt(self.goals, memory, self.actions)

            print("Agent thinking...")
            # Generate a response from the agent
            response = self.prompt_llm_for_action(prompt)
            print(f"Agent Decision: {response}")

            # Determine which action the agent wants to execute
            action, invocation = self.get_action(response)

            # Execute the action in the environment
            result = self.environment.execute_action(action, invocation["args"])
            print(f"Action Result: {result}")

            # Update the agent's memory with information about what happened
            self.update_memory(memory, response, result)

            # Check if the agent has decided to terminate
            if self.should_terminate(response):
                break

        return memory

# Main Code

In [35]:
# First, we'll define our tools using decorators
@register_tool(tags=["file_operations", "read"])
def read_project_file(name: str) -> str:
    """Reads and returns the content of a specified project file.

    Opens the file in read mode and returns its entire contents as a string.
    Raises FileNotFoundError if the file doesn't exist.

    Args:
        name: The name of the file to read

    Returns:
        The contents of the file as a string
    """
    with open(name, "r") as f:
        return f.read()

@register_tool(tags=["file_operations", "list"])
def list_project_files() -> List[str]:
    """Lists all Python files in the current project directory.

    Scans the current directory and returns a sorted list of all files
    that end with '.py'.

    Returns:
        A sorted list of Python filenames
    """
    return sorted([file for file in os.listdir(".")
                  if file.endswith(".py")])

@register_tool(tags=["system"], terminal=True)
def terminate(message: str) -> str:
    """Terminates the agent's execution with a final message.

    Args:
        message: The final message to return before terminating

    Returns:
        The message with a termination note appended
    """
    return f"{message}\nTerminating..."



# Define the agent's goals
goals = [
    Goal(priority=1,
          name="Gather Information",
          description="Read each file in the project in order to build a deep understanding of the project in order to write a README"),
    Goal(priority=1,
          name="Terminate",
          description="Call terminate when done and provide a complete README for the project in the message parameter")
]

# Create an agent instance with tag-filtered actions

agent = Agent(
    goals=goals,
    agent_language=AgentFunctionCallingActionLanguage(),
    # The ActionRegistry now automatically loads tools with these tags
    action_registry=PythonActionRegistry(tags=["file_operations", "system"]),
    generate_response=generate_response,
    environment=Environment()
)

# Run the agent with user input
user_input = "Write a README for this project."
final_memory = agent.run(user_input)
print(final_memory.get_memories())

Agent thinking...
DEBUG: Starting generate_response
DEBUG: Prompt has 2 messages and 3 tools
DEBUG: Cohere model initialized
DEBUG: Converted 2 messages
DEBUG: Tools present, setting up tool calling
DEBUG: Original tools: [{'type': 'function', 'function': {'name': 'read_project_file', 'description': "Reads and returns the content of a specified project file.\n\n    Opens the file in read mode and returns its entire contents as a string.\n    Raises FileNotFoundError if the file doesn't exist.\n\n    Args:\n        name: The name of the file to read\n\n    Returns:\n        The contents of the file as a string", 'parameters': {'type': 'object', 'properties': {'name': {'type': 'string'}}, 'required': ['name']}}}, {'type': 'function', 'function': {'name': 'list_project_files', 'description': "Lists all Python files in the current project directory.\n\n    Scans the current directory and returns a sorted list of all files\n    that end with '.py'.\n\n    Returns:\n        A sorted list of 